# Preparing the dataset from stock and news data
We are using the regression method for finding value of closing value after 15, 30, 45, 60 minutes when the news breaks out.
We are cleaning the news title by removing basic punctuation by using the __cleanString__ method. 

Since the format of the time in stock data and news file are in different format, we are converting them into one format using the __convert__ method. Here we are converting the format of time of news into time of stock data.

Now we know that the Indian stock market has a running time from 0915 hrs to 1530 hrs but the news will break out 24 hours. So we need to handle the news coming out of the market hours. This is done by using the __check__ method. If a news is coming after the market hours, we redirect that news to 0915 hrs next day.

We are maintaining a lists of 15, 30, 45 and 60 values. Now consider that a news breaks out at a particular time. Now the time of the news is checked with the time of the stock data. When the time in the stock data is found, then it stores the next 30 values (respectively the 15, 45 and 60 values) into the list. Now using stats.lineregress, we get the slope and intercept which is used to get the final value. Once we get the final value, we can get the percentage change from the initial value.

The label calculation is done on the 30 minute percentage change value as well as the 45 minute percentage change value. This gives us two labels to play with to determine which gives us a better approximation.

For scaling the data to include all the news and quant files, we apply os.walk method to walk into particular news and quant folders and read all files one by one and perform the above mentioned tasks.

In [1]:
import csv
import pandas as pd
import re
import os
import datetime
import numpy as np
import scipy.stats as stats

def cleanString(incomingString):
    newstring = incomingString
    newstring = newstring.replace("!","")
    newstring = newstring.replace("@","")
    newstring = newstring.replace("#","")
    newstring = newstring.replace("$","")
    newstring = newstring.replace("%","")
    newstring = newstring.replace("^","")
    newstring = newstring.replace("&","and")
    newstring = newstring.replace("*","")
    newstring = newstring.replace("(","")
    newstring = newstring.replace(")","")
    newstring = newstring.replace("+","")
    newstring = newstring.replace("=","")
    newstring = newstring.replace("?","")
    newstring = newstring.replace("\'","")
    newstring = newstring.replace("\"","")
    newstring = newstring.replace("{","")
    newstring = newstring.replace("}","")
    newstring = newstring.replace("[","")
    newstring = newstring.replace("]","")
    newstring = newstring.replace("<","")
    newstring = newstring.replace(">","")
    newstring = newstring.replace("~","")
    newstring = newstring.replace("`","")
    newstring = newstring.replace(":","")
    newstring = newstring.replace(";","")
    newstring = newstring.replace("|","")
    newstring = newstring.replace("\\","")
    newstring = newstring.replace("/","")
    newstring = newstring.replace(",","")
    newstring = newstring.replace(".","")
    newstring = newstring.lower()
    return newstring


def convert(time):
    test = time

    test1 = pd.to_datetime(test)
    test2 = str(test1)
    test2 = test2.replace(" ","T")
    return test2

def check(time):
    try:
        if (((int(time[11:13])==9) and (int(time[14:16])<15))) or (int(time[11:13])<9):
            #print('if')
            time = time[:11] + '09:15:00'
        
        elif (int(time[11:13]) >= 14 and int(time[14:16])>59):
            #print('else')
            st = time
            parts = st.split('T')
            date = datetime.datetime(int(st[0:4]),int(st[5:7]),int(st[8:10]),00,00,00)
            date += datetime.timedelta(days=1)
            part = str(date).split(' ' )
            time = part[0]+"T"+ '09:15:00'        
    except:
        print ("except")
        pass
    return time
def prepareDataset(readNews,l):
    try:
                for n in readNews:
                    
                        file = open (r'C:\Users\T\Desktop\Saber\Merging quant and news\Preparing dataset\Quant\\'+l,'r',encoding='utf-8')
                        string1=""
                        X1 = [i for i in range(1,31)]
                        X2 = [j for j in range(1,16)]
                        X3 = [k for k in range(1,46)]
                        X4 = [l for l in range(1,61)]
                        for s1 in re.split(' ',n[5]):
                            string1 = string1 + " "+cleanString(s1)
                        count =0
                        flag=0
                        #timeWithT=convert(n[3])
                        #print(timeWithT)
                        temptime=str(check(n[3]))+'+0530'
                        print(temptime)
                        Y = []
                        Z=[]
                        W=[]
                        V=[]
                        YV = []
                        ZV=[]
                        WV=[]
                        VV=[]
                        rea = (csv.reader(file))
                        next(rea,None)
                        for q in rea:
                                if(count==0 and str(q[1])==temptime):
                                    flag=1           
                                if(flag == 1):                    
                                    count +=1
                                    if(count<=15):
                                        Z.append(q[5])
                                        quant15=q[5]
                                        ZV.append(q[6])
                                    if(count <= 30):
                                        Y.append(q[5])
                                        quant30=q[5]
                                        YV.append(float(q[6]))
                                    if(count<=45):
                                        W.append(q[5])
                                        quant45=q[5]
                                        WV.append(float(q[6]))
                                    if(count<=60):
                                        V.append(q[5])
                                        quant60=q[5]
                                        VV.append(q[6])
                                if (count == 60):
                                    break
                        file.close()
                        #print ("DATA_START")
                        if (len(Y) == 30 or len(Z)==15 or len(W)==45 or len(V)==60):
                            #for s in Y:
                                #print(s)
                            slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(X1, np.array(Y).astype(np.float))
                            slope2, intercept2, r_value2, p_value2, std_err2 = stats.linregress(X2, np.array(Z).astype(np.float))
                            slope3, intercept3, r_value3, p_value3, std_err3 = stats.linregress(X3, np.array(W).astype(np.float))
                            slope4, intercept4, r_value4, p_value4, std_err4 = stats.linregress(X4, np.array(V).astype(np.float))
                            slope5, intercept5, r_value5, p_value5, std_err5 = stats.linregress(X1, np.array(YV).astype(np.float))
                            slope6, intercept6, r_value6, p_value6, std_err6 = stats.linregress(X2, np.array(ZV).astype(np.float))
                            slope7, intercept7, r_value7, p_value7, std_err7 = stats.linregress(X3, np.array(WV).astype(np.float))
                            slope8, intercept8, r_value8, p_value8, std_err8 = stats.linregress(X4, np.array(VV).astype(np.float))
                            percentage30 = ((intercept1+slope1*30)-float(Y[0]))*100/float(Y[0])
                            percentINTO1030 = percentage30*10
                            percentage15 = ((intercept2+slope2*15)-float(Z[0]))*100/float(Z[0])
                            percentINTO1015 = percentage15*10
                            percentage45 = ((intercept3+slope3*45)-float(W[0]))*100/float(W[0])
                            percentINTO1045 = percentage45*10
                            percentage60 = ((intercept4+slope4*60)-float(V[0]))*100/float(V[0])
                            percentINTO1060 = percentage60*10
                                   
                            print(p_value1)
                            
                            percentage1530=(float(quant30)-float(quant15))*100/float(quant15)
                            percentage1545=(float(quant45)-float(quant15))*100/float(quant15)
                            percentage1560=(float(quant60)-float(quant15))*100/float(quant15)
                            percentage3045=(float(quant45)-float(quant30))*100/float(quant30)
                            percentage3060=(float(quant60)-float(quant30))*100/float(quant30)
                            percentage4560=(float(quant60)-float(quant45))*100/float(quant45)
                            
                            percentage30V = slope5
                            #percentINTO1030 = percentage30*10
                            percentage15V = ((intercept6+slope6*15)-float(ZV[0]))*100/float(ZV[0])
                            #percentINTO1015 = percentage15*10
                            percentage45V = slope7
                            #percentINTO1045 = percentage45*10
                            percentage60V = ((intercept8+slope8*60)-float(VV[0]))*100/float(VV[0])
                            percentINTO1060 = percentage60*10
                            #print (percentINTO10)
                            if(float(percentage30) > 1):
                                label1=2
                            elif(float(percentage30) > 0.5 and float(percentage30)<1):
                                label1=1
                            elif(float(percentage30) < -1):
                                label1=-2
                            elif(float(percentage30) > -0.5 and float(percentage30)<0):
                                label1=-1
                            else:
                                label1=0
                                
                            if(float(percentage45) > 1):
                                label2=2
                            elif(float(percentage45) > 0.5 and float(percentage45)<1):
                                label2=1
                            elif(float(percentage45) < -1):
                                label2=-2
                            elif(float(percentage45) > -0.5 and float(percentage45)<0):
                                label2=-1
                            else:
                                label2=0
                            if(flag==1):
                                stddev30=np.std(YV)
                                stddev45=np.std(WV)
                                csvwriter.writerow([temptime,string1,n[7],n[8],n[0],n[1],n[2],n[6],q[5],q[6],str(percentage30),str(percentage45),str(percentage30V),str(percentage45V),label1,label2,stddev30,stddev45])

    except:
                newfile.close()
                print("KeyBoard Interupt")

newfile = open(r'dataset.csv','w',newline='',encoding='utf-8')
csvwriter=csv.writer(newfile)
csvwriter.writerow(['Date','Title','Subtitle','Content','CompanyName','Source','Category','Tags','Close','Volume','PC030','PC045','VolumeChange30','VolumeChange45','Label1','Label2','Stddev30','Stddev45'])

quant_file= [(x[2]) for x in os.walk(r'C:\Users\T\Desktop\Saber\Merging quant and news\Preparing dataset\Quant')]
q = quant_file[0] #q is a list containing names of all files in quant
news_files = [(y[2]) for y in os.walk(r'C:\Users\T\Desktop\Saber\Merging quant and news\Preparing dataset\News')]
s = news_files[0] #s is a list containing names of all files in news
for j in s:
    
    for l in q:
        m = j.lower()[:-4]
        n = l.lower()[:-4]
        print (m,n)
        if m in n:
            news = open(r"C:\Users\T\Desktop\Saber\Merging quant and news\Preparing dataset\News\\"+ j,'r',encoding ="ISO-8859-1")
            readNews = csv.reader(news)
            next(readNews, None)
            print (l)
            prepareDataset(readNews,l)

newfile.close()

bpcl bpcl
BPCL.csv
04-01-2013 11:34+0530
04-01-2013 12:13+0530
04-01-2013 16:50+0530
07-01-2013 11:50+0530
07-01-2013 16:43+0530
07-01-2013 19:43+0530
10-01-2013 10:42+0530
17-01-2013 15:37+0530
18-01-2013 12:19+0530
18-01-2013 15:26+0530
21-01-2013 14:41+0530
04-02-2013 09:15:00+0530
04-02-2013 09:21+0530
08-02-2013 15:03+0530
14-02-2013 09:15:00+0530
15-02-2013 13:02+0530
20-02-2013 14:18+0530
08-03-2013 11:05+0530
08-03-2013 12:21+0530
08-03-2013 15:02+0530
11-03-2013 09:15:00+0530
25-03-2013 09:56+0530
25-03-2013 13:52+0530
25-03-2013 16:29+0530
28-03-2013 09:15:00+0530
29-03-2013 18:33+0530
15-04-2013 10:32+0530
15-04-2013 13:55+0530
17-04-2013 19:02+0530
22-04-2013 11:28+0530
22-04-2013 11:32+0530
22-04-2013 17:43+0530
23-04-2013 09:15:00+0530
24-04-2013 16:57+0530
03-05-2013 14:11+0530
06-05-2013 09:15:00+0530
10-05-2013 09:15:00+0530
10-05-2013 13:18+0530
11-05-2013 11:30+0530
11-05-2013 11:56+0530
14-05-2013 11:16+0530
21-05-2013 19:55+0530
22-05-2013 10:54+0530
22-05-2013 12:

28-05-2016 16:26+0530
02-06-2016 13:37+0530
02-06-2016 14:22+0530
03-06-2016 14:38+0530
07-06-2016 16:45+0530
08-06-2016 19:35+0530
17-06-2016 14:15+0530
20-06-2016 15:06+0530
22-06-2016 09:25+0530
24-06-2016 09:15:00+0530
27-06-2016 14:17+0530
28-06-2016 09:15:00+0530
01-07-2016 15:10+0530
04-07-2016 13:22+0530
08-07-2016 09:16+0530
11-07-2016 15:31+0530
29-07-2016 22:32+0530
01-08-2016 09:49+0530
03-08-2016 09:15:00+0530
04-08-2016 13:21+0530
08-08-2016 13:50+0530
20-08-2016 16:59+0530
29-08-2016 13:54+0530
31-08-2016 19:48+0530
01-09-2016 09:15:00+0530
02-09-2016 14:55+0530
06-09-2016 14:09+0530
14-09-2016 09:15:00+0530
23-09-2016 09:15:00+0530
23-09-2016 20:07+0530
04-01-2017 19:33+0530
09-01-2017 20:22+0530
10-01-2017 15:16+0530
12-01-2017 09:15:00+0530
07-02-2017 12:16+0530
09-02-2017 12:40+0530
14-02-2017 14:24+0530
17-02-2017 14:30+0530
17-02-2017 21:41+0530
20-02-2017 13:51+0530
22-02-2017 09:15:00+0530
22-02-2017 09:15:00+0530
22-02-2017 11:49+0530
02-03-2017 13:22+0530
14-03

16-06-2015 12:01+0530
16-06-2015 09:55+0530
04-06-2015 11:18+0530
01-06-2015 16:22+0530
29-05-2015 17:11+0530
28-05-2015 14:47+0530
25-05-2015 10:39+0530
22-05-2015 17:06+0530
12-05-2015 14:45+0530
11-05-2015 15:11+0530
01-05-2015 10:55+0530
22-04-2015 12:27+0530
17-04-2015 10:38+0530
13-04-2015 14:21+0530
08-04-2015 11:37+0530
26-03-2015 17:04+0530
18-03-2015 14:25+0530
16-02-2015 10:10+0530
14-02-2015 10:51+0530
13-02-2015 14:18+0530
13-02-2015 14:00+0530
10-02-2015 18:37+0530
03-02-2015 10:30+0530
02-02-2015 21:40+0530
29-01-2015 11:03+0530
29-01-2015 09:15:00+0530
28-01-2015 11:29+0530
16-01-2015 09:53+0530
15-01-2015 14:02+0530
09-01-2015 14:29+0530
26-12-2014 09:16+0530
26-12-2014 09:15:00+0530
13-11-2014 11:45+0530
13-11-2014 10:54+0530
12-11-2014 09:20+0530
16-10-2014 12:29+0530
08-10-2014 16:48+0530
08-10-2014 09:26+0530
08-09-2014 13:56+0530
21-02-2013 09:15:00+0530
21-09-2013 22:30+0530
23-09-2013 09:15:00+0530
28-09-2013 09:15:00+0530
13-11-2013 22:44+0530
01-09-2014 09:15:

2014-10-29T09:19:00+0530
2014-10-29T09:42:00+0530
2014-10-29T12:40:00+0530
2014-10-30T11:35:00+0530
2014-10-30T18:56:00+0530
2014-11-02T14:41:00+0530
2014-11-13T21:42:00+0530
2014-11-14T17:13:00+0530
2014-11-14T17:13:00+0530
2014-11-19T09:18:00+0530
2014-11-19T09:15:00+0530
2014-11-20T12:53:00+0530
2014-11-20T18:13:00+0530
2014-11-21T09:15:00+0530
2014-11-21T11:12:00+0530
2014-11-26T09:15:00+0530
2014-11-26T11:15:00+0530
2014-11-26T12:42:00+0530
2014-11-27T17:55:00+0530
2014-12-03T15:47:00+0530
2014-12-03T16:10:00+0530
2014-12-03T15:47:00+0530
2014-12-03T16:10:00+0530
2014-12-04T15:26:00+0530
2014-12-03T15:47:00+0530
2014-12-03T16:10:00+0530
2014-12-04T15:26:00+0530
2014-12-10T17:26:00+0530
2014-12-15T10:27:00+0530
2014-12-15T12:10:00+0530
2014-12-16T11:39:00+0530
2014-12-17T12:23:00+0530
2014-12-17T13:09:00+0530
2014-12-17T15:31:00+0530
2014-12-17T22:46:00+0530
2014-12-18T09:34:00+0530
2014-12-23T16:35:00+0530
2015-01-09T10:24:00+0530
2015-01-09T20:12:00+0530
2015-01-10T12:50:00+0530


2016-12-28T12:51:00+0530
2016-12-29T09:15:00+0530
2017-01-09T09:15:00+0530
2017-01-09T15:17:00+0530
2017-02-03T13:07:00+0530
2017-02-06T12:25:00+0530
2017-02-07T12:21:00+0530
2017-02-07T19:21:00+0530
2017-02-08T16:59:00+0530
2017-02-08T16:59:00+0530
2017-02-09T11:00:00+0530
2017-02-17T17:45:00+0530
2017-02-21T14:41:00+0530
2017-03-02T13:44:00+0530
2017-03-06T09:15:00+0530
2017-03-06T09:15:00+0530
2017-03-06T12:53:00+0530
2017-03-10T09:15:00+0530
2017-03-10T09:15:00+0530
2017-03-31T09:15:00+0530
2017-04-12T12:20:00+0530
2017-04-13T11:47:00+0530
2017-04-28T23:59:00+0530
2017-05-25T16:53:00+0530
2017-05-26T09:15:00+0530
2017-05-26T16:08:00+0530
2017-05-26T11:34:00+0530
2017-05-27T10:37:00+0530
2017-05-25T16:53:00+0530
2017-05-26T09:15:00+0530
2017-05-26T16:08:00+0530
2017-05-26T11:34:00+0530
2017-05-27T10:37:00+0530
2017-05-25T16:53:00+0530
2017-05-26T09:15:00+0530
2017-05-26T16:08:00+0530
2017-05-26T11:34:00+0530
2017-05-27T10:37:00+0530
2017-06-13T19:02:00+0530
2017-06-14T15:18:00+0530


2016-07-23T09:15:00+0530
2016-07-16T09:15:00+0530
2016-07-14T12:36:00+0530
2016-07-08T09:49:00+0530
2016-07-08T10:44:00+0530
2016-06-19T17:17:00+0530
2016-06-19T10:12:00+0530
2016-06-08T10:45:00+0530
2016-06-08T09:15:00+0530
2016-06-06T09:15:00+0530
2016-06-06T09:15:00+0530
2016-05-27T11:24:00+0530
2016-05-25T11:01:00+0530
2016-05-25T12:35:00+0530
2016-05-25T09:15:00+0530
2016-05-25T09:15:00+0530
2016-05-24T16:05:00+0530
2016-05-20T09:15:00+0530
2016-05-02T09:15:00+0530
2016-04-06T09:15:00+0530
2016-03-28T09:15:00+0530
2016-03-18T09:15:00+0530
2016-03-16T09:15:00+0530
2016-03-16T09:15:00+0530
2014-11-27T13:38:00+0530
0.012418272233213701
2014-11-27T10:24:00+0530
0.00020731868800961354
2014-11-27T09:15:00+0530
2.00200342064972e-12
2014-11-26T09:15:00+0530
2014-11-25T09:15:00+0530
2014-11-25T11:05:00+0530
2014-11-24T09:15:00+0530
2014-11-21T12:35:00+0530
2014-11-21T09:23:00+0530
2014-11-21T09:15:00+0530
2014-11-21T09:23:00+0530
2014-11-20T09:15:00+0530
2014-11-20T09:15:00+0530
2014-11-18